# Evaluation LDA MAX

- kombi of different class und topic
- mapping averbis klasse mit meiner Zahl
- Fehlerrrberechnung - erhalte Zahl ob korrekt oder nicht

In [1]:
# imports
import pandas as pd
import numpy as np
import itertools
from itertools import permutations
from sklearn import metrics

In [2]:
#read csv file from predicted topics
df = pd.read_csv('/home/ubuntu/ullrich/my_code/data/predicted_LDA_määäx.csv', sep=',')

In [3]:
# drop unnessecary columns 
df = df.drop(['Unnamed: 0'], axis=1)
# replace topic name
df = df.replace({'class':'ErnÃ¤hrung'},{'class':'Ernährung'})

In [4]:
df

,dbrecordid,class,combined,topic,certainty
0,BASE::ftdoajarticles:oai:doaj.org/article:d7b5...,Ernährung,"type,diabetes,reported,greater,prevalence,poor...",3.0,0.946439
1,M3972220,Medizin,"document,case,acute,nonobstructive,appendiciti...",3.0,0.975151
2,AGRISUS201500009429,Umweltwissenschaften,"study,examine,biological,process,involved,ammo...",3.0,0.733459
3,M32650928,Medizin,"highly,reliable,identification,adult,hypertrop...",1.0,0.674510
4,M6641809,Medizin,"sedimentation,study,colchicine,binding,assay,d...",1.0,0.975639
...,...,...,...,...,...
99265,BASE::ftdoajarticles:oai:doaj.org/article:12cf...,Umweltwissenschaften,"stock,assessment,edible,bivalve,solen,brevis,w...",NaN,NaN
99266,M2887402,Landwirtschaft,"report,describes,localization,kda,glycoprotein...",3.0,0.975730
99267,M34390914,Landwirtschaft,"systematic,knowledge,role,dna,repair,gene,leve...",3.0,0.508172
99268,AGRICOLAIND607823800,Medizin,"background,depression,global,disease,affect,ph...",NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96736 entries, 0 to 96735
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dbrecordid  96736 non-null  object 
 1   class       96736 non-null  object 
 2   topic       70615 non-null  float64
 3   certainty   70615 non-null  float64
dtypes: float64(2), object(2)
memory usage: 3.0+ MB


- 26.000 none values ?? - Text beinhaltet kein Wort aus der Themenklasse - Beschiss
- unter 60 % raus 

In [4]:
#df_test = df[(df['certainty'] is not None) & (df['certainty'] >= 0.6 )]

In [5]:
#define all possible combinations of the classes
classes = ['Medizin', 'Landwirtschaft', 'Umweltwissenschaften', 'Ernährung']
topics = [0.0,1.0,2.0,3.0]

unique_combinations = []
permut = itertools.permutations(classes, len(topics))

for comb in permut:
    zipped = zip(comb, topics)
    unique_combinations.append(list(zipped))
 
# printing unique_combination list
print(unique_combinations)

[[('Medizin', 0.0), ('Landwirtschaft', 1.0), ('Umweltwissenschaften', 2.0), ('Ernährung', 3.0)], [('Medizin', 0.0), ('Landwirtschaft', 1.0), ('Ernährung', 2.0), ('Umweltwissenschaften', 3.0)], [('Medizin', 0.0), ('Umweltwissenschaften', 1.0), ('Landwirtschaft', 2.0), ('Ernährung', 3.0)], [('Medizin', 0.0), ('Umweltwissenschaften', 1.0), ('Ernährung', 2.0), ('Landwirtschaft', 3.0)], [('Medizin', 0.0), ('Ernährung', 1.0), ('Landwirtschaft', 2.0), ('Umweltwissenschaften', 3.0)], [('Medizin', 0.0), ('Ernährung', 1.0), ('Umweltwissenschaften', 2.0), ('Landwirtschaft', 3.0)], [('Landwirtschaft', 0.0), ('Medizin', 1.0), ('Umweltwissenschaften', 2.0), ('Ernährung', 3.0)], [('Landwirtschaft', 0.0), ('Medizin', 1.0), ('Ernährung', 2.0), ('Umweltwissenschaften', 3.0)], [('Landwirtschaft', 0.0), ('Umweltwissenschaften', 1.0), ('Medizin', 2.0), ('Ernährung', 3.0)], [('Landwirtschaft', 0.0), ('Umweltwissenschaften', 1.0), ('Ernährung', 2.0), ('Medizin', 3.0)], [('Landwirtschaft', 0.0), ('Ernährung',

In [6]:
#get lists of pred and true values
pred_test = df['topic'].values.tolist()
true_test = df['class'].values.tolist()

In [7]:
#replace items in predicted list
def replace_items(pred_list, true_list):
    liste = list(pred_list)
    for i in range(len(liste)):
        for tupel in true_list:
            if liste[i] == tupel[1]:
                liste[i] = tupel[0]
    return liste

In [8]:
#create f1-score for every combination
def f1_score(combinations, pred_list, true_list):
    result = []
    for combi in combinations:
        res = {}
        pred = replace_items(pred_list,combi)
        f1 = metrics.f1_score(true_list, pred, average='weighted')
        res.update({'Combi': combi,'f1_score': f1})
        result.append(res)
    return result

In [9]:
#get dictionary for each combination and every score
res_dict = f1_score(unique_combinations, pred_test, true_test)

In [10]:
#get highest value of dictionary
def highest_val(lst, key, key2):
    highest_value = None
    for dict in lst:
        score = dict[key]
        combi = dict[key2]  
        if highest_value is None or score > highest_value:
            highest_value = score
            pred_combi = combi
    return highest_value, pred_combi

In [11]:
f1 = highest_val(res_dict,'f1_score','Combi')

In [12]:
f1

(0.182059591509643,
 [('Landwirtschaft', 0.0),
  ('Umweltwissenschaften', 1.0),
  ('Ernährung', 2.0),
  ('Medizin', 3.0)])

In [ ]:
#get list of the class and predicted topic
pred_averbis = df[['class', 'topic']].values.tolist()
pred_averbis_test = df_test[['class', 'topic']].values.tolist()

In [ ]:
def calculate_acc_score(pred_class, combinations):
    result = [] 
    for combi in combinations:
        tmp = {}
        total_count = len(pred_class)
        match_count = 0
        #print(combi)
        for item in pred_class:
            item = tuple(item)
            if item in combi:
                match_count = match_count + 1
        #print(match_count)        
        accuracy = match_count / total_count
        tmp.update({'Combi': combi,'match_count': match_count, 'acc_score': accuracy})
        result.append(tmp)
    return result

In [ ]:
#get dict of results
result = calculate_acc_score(pred_averbis, unique_combinations)
result_test = calculate_acc_score(pred_averbis_test, unique_combinations)

In [ ]:
highest_val(result, 'acc_score', 'Combi')
highest_val(result_test, 'acc_score', 'Combi')

(0.23706789355881622,
 [('Ernährung', 0.0),
  ('Landwirtschaft', 1.0),
  ('Rest', 2.0),
  ('Medizin', 3.0),
  ('Umweltwissenschaften', 4.0)])